###### Visualización de Datos - Práctica 1

# Preparación de los datos

In [6]:
#Imports
import pandas as pd
import numpy as np
import os

In [7]:
print(os.getcwd())

C:\Users\Usuario\Visualizacion_Datos_PRA_1\notebooks


In [8]:
#Carga datasets
salaries_path = './data/raw/salaries.csv'
cost_of_living_path = './data/raw/cost_of_living.csv'
iso_country_codes_path = './data/raw/iso_country_codes.csv'

df_salaries = pd.read_csv(salaries_path)
df_cost_of_living = pd.read_csv(cost_of_living_path, delimiter=';')
df_iso_country_codes = pd.read_csv(iso_country_codes_path, delimiter=';')

FileNotFoundError: [Errno 2] No such file or directory: './data/raw/salaries.csv'

In [ ]:
df_salaries.sample(3)

In [ ]:
df_cost_of_living.sample(3)

In [ ]:
df_iso_country_codes.sample(3)

Para poder combinar la información de los dataframes y pensando en mostrar los nombres enteros de los países en las visualizaciones sustituimos los códigos de las columnas "employee_residence' y 'company_location' por el nombre del país completo:

In [ ]:
# Diccionario código - nombre país
iso_mapping = df_iso_country_codes.set_index('Alpha-2 code')['Country'].to_dict()

# Cambiar los códigos por los nombres
df_salaries['employee_residence'] = df_salaries['employee_residence'].map(iso_mapping)
df_salaries['company_location'] = df_salaries['company_location'].map(iso_mapping)


df_salaries.sample(10)

Combinamos los datos para crear un único archivo con el que crearemos las visualizaciones:

In [ ]:
df_salaries = df_salaries.merge(df_cost_of_living, left_on="employee_residence", right_on="Country", how="left")

In [ ]:
df_salaries.info()

In [ ]:
# Eliminamos las columnas que ya no queremos
df_salaries = df_salaries.drop(columns=['Rank','Country'])

# Renombramos las que necesitamos para el análisis
df_salaries.columns = df_salaries.columns.str.lower().str.replace(' ', '_')

# Eliminamos los 4 registros con cost_of_living_index nulo, de países 
df_salaries = df_salaries[df_salaries['cost_of_living_index'].notna()]

In [ ]:
df_salaries

In [ ]:
# Reemplazar las comas por puntos en las columnas relevantes
df_salaries['cost_of_living_index'] = df_salaries['cost_of_living_index'].str.replace(',', '.')

# Convertir la columna a float
df_salaries['cost_of_living_index'] = df_salaries['cost_of_living_index'].astype(float)

# Repetir para las otras columnas con el mismo problema
df_salaries['rent_index'] = df_salaries['rent_index'].str.replace(',', '.').astype(float)
df_salaries['cost_of_living_plus_rent_index'] = df_salaries['cost_of_living_plus_rent_index'].str.replace(',', '.').astype(float)
df_salaries['groceries_index'] = df_salaries['groceries_index'].str.replace(',', '.').astype(float)
df_salaries['restaurant_price_index'] = df_salaries['restaurant_price_index'].str.replace(',', '.').astype(float)
df_salaries['local_purchasing_power_index'] = df_salaries['local_purchasing_power_index'].str.replace(',', '.').astype(float)

In [ ]:
# Filtrar por los años 2024 y 2025
df_salaries_filtered_years = df_salaries[df_salaries['work_year'].isin([2024, 2025])]

# Contar los registros por país de residencia en los años 2024 y 2025
country_counts = df_salaries_filtered_years['employee_residence'].value_counts()

# Mostrar los países con su respectivo número de registros para los años 2024 y 2025
country_counts.head()

In [ ]:
# Definir el umbral de registros mínimos
umbral = 10

# Filtrar los países con al menos 'umbral' registros en los años 2024 y 2025
countries_with_enough_data = country_counts[country_counts >= umbral].index

# Filtrar el dataframe original para quedarse solo con estos países y años
df_salaries_filtered = df_salaries_filtered_years[df_salaries_filtered_years['employee_residence'].isin(countries_with_enough_data)]

df_salaries_filtered

In [ ]:
# Añadir la relación salario/costo de vida
df_salaries['salario_costo_vida'] = df_salaries['salary_in_usd'] / df_salaries['cost_of_living_index'].astype(float)

# Añadir el salario ajustado por poder adquisitivo
df_salaries['salario_ajustado'] = df_salaries['salary_in_usd'] * (df_salaries['local_purchasing_power_index'].astype(float) / 100)

# Agrupar por país y calcular la media de las métricas
df_pais_avg = df_salaries.groupby('employee_residence')[['salario_costo_vida', 'salario_ajustado']].mean().reset_index()

# Ver las primeras filas del nuevo dataset
df_pais_avg.head(25)

In [ ]:
# Guardar df_salaries_filtered en CSV
df_salaries_filtered.to_csv('../data/clean/df_salaries_clean.csv', index=False)

# Guardar df_pais_avg en CSV
df_pais_avg.to_csv('../data/clean/df_pais_avg.csv', index=False)